In [ ]:
import pandas as pd
import io

nelex_df = pd.read_csv('nelex_NGA.csv')
nelex_df.head()

In [ ]:
microfinance_df = pd.read_csv("microfinance_NGA.csv")
microfinance_df.head()

In [ ]:
!pip install geopy geopandas plotly
from geopy.geocoders import Nominatim
import geopandas as gpd
import plotly.express as px
from shapely.geometry import Point


In [ ]:
# Initialize geolocator
geolocator = Nominatim(user_agent="nigeria_map")

# Function to geocode addresses
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return None, None

# Geocode NELEX office addresses
nelex_df[['latitude', 'longitude']] = nelex_df['nelex_address'].apply(
    lambda x: pd.Series(geocode_address(x))
)

# Geocode microfinance bank addresses
microfinance_df[['latitude', 'longitude']] = microfinance_df['microfinance_address'].apply(
    lambda x: pd.Series(geocode_address(x))
)

# Drop any rows where geocoding failed
nelex_df.dropna(subset=['latitude', 'longitude'], inplace=True)
microfinance_df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Convert to GeoDataFrames
nelex_df['geometry'] = nelex_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
nelex_gdf = gpd.GeoDataFrame(nelex_df, geometry='geometry')

microfinance_df['geometry'] = microfinance_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
microfinance_gdf = gpd.GeoDataFrame(microfinance_df, geometry='geometry')

In [ ]:
# Load Nigeria shapefile
nigeria_gdf = gpd.read_file("shapefile_NGA_shp")

In [ ]:
# Convert geometries to latitude and longitude for Plotly
nelex_df['lon'] = nelex_df['geometry'].apply(lambda point: point.x)
nelex_df['lat'] = nelex_df['geometry'].apply(lambda point: point.y)

microfinance_df['lon'] = microfinance_df['geometry'].apply(lambda point: point.x)
microfinance_df['lat'] = microfinance_df['geometry'].apply(lambda point: point.y)

# Plot using Plotly
fig = px.scatter_mapbox(
    nigeria_gdf,  # Use nigeria_gdf for base map
    lat=nigeria_gdf.geometry.centroid.y,  # Latitude for Nigeria (using centroid)
    lon=nigeria_gdf.geometry.centroid.x,  # Longitude for Nigeria (using centroid)
    color_discrete_sequence=["green"],  # Distinct color for Nigeria
    center={"lat": 9.0820, "lon": 8.6753},  # Center on Nigeria
    zoom=4,
    mapbox_style="carto-positron",  # Setting map style
    hover_data=['COUNTRY'] # Display the name of Nigeria on hover
)

# Add NELEX offices to the map as a separate trace
fig.add_trace(
    px.scatter_mapbox(
        nelex_df,
        lat="lat",
        lon="lon",
        color_discrete_sequence=["blue"], # Distinct color for NELEX markers
        hover_data=['nelex_office'] # Display NELEX center name on hover
    ).data[0]
)

# Add microfinance banks to the map as a separate trace
fig.add_trace(
    px.scatter_mapbox(
        microfinance_df,
        lat="lat",
        lon="lon",
        color_discrete_sequence=["red"], # Distinct color for microfinance markers
        hover_data=['microfinance_bank'] # Display microfinance bank name on hover
    ).data[0]
)

# Update layout for better visualization
fig.update_layout(
    title="Locations of NELEX Centers and Microfinance Banks in Nigeria",
    mapbox=dict(
        zoom=4.65
    ),
    legend=dict(
        title="Legend",
        orientation="h"
    )
)

fig.show()